In [2]:
# Neuronal Network 1 for learning the implied vola 
# source: https://github.com/amuguruza/NN-StochVol-Calibrations/blob/master/1Factor/Flat%20Forward%20Variance/NN1Factor.ipynb 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import InputLayer,Dense,Dropout,Conv2D,MaxPooling2D,ZeroPadding2D,Flatten
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import time
import scipy

# Data Import
import scipy.io
mat         = scipy.io.loadmat('data_price_maxbounds_5000_0005_09_11_30_210.mat')
data        = mat['data_price']
Nparameters = 5
maturities  = np.array([30, 60, 90, 120, 150, 180, 210])
strikes     = np.array([0.9, 0.925, 0.95, 0.975, 1.0, 1.025, 1.05, 1.075, 1.1])
Nstrikes    = len(strikes)   
Nmaturities = len(maturities)   
xx          = data[:,:Nparameters]
yy          = data[:,Nparameters+2:]


# split into train and test sample
X_train, X_test, y_train, y_test = train_test_split(
    xx, yy, test_size=0.15)#, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
   X_train, y_train, test_size=0.15)#, random_state=42)

Ntest= X_test.shape[0]
Ntrain= X_train.shape[0]
Nval= X_val.shape[0]
scale=StandardScaler()
y_train_transform = scale.fit_transform(y_train)
y_val_transform   = scale.transform(y_val)
y_test_transform  = scale.transform(y_test)
def ytransform(y_train,y_val,y_test):
    #return [scale.transform(y_train),scale.transform(y_val), 
    #        scale.transform(y_test)]
    return [y_train,y_val,y_test]
def yinversetransform(y):
    return y
    #return scale.inverse_transform(y)
[y_train_trafo, y_val_trafo, y_test_trafo]=ytransform(y_train, y_val, y_test)
ub=np.amax(xx, axis=0)
lb=np.amin(xx, axis=0)
def myscale(x):
    res=np.zeros(Nparameters)
    for i in range(Nparameters):
        res[i]=(x[i] - (ub[i] + lb[i])*0.5) * 2 / (ub[i] - lb[i])
    return res
def myinverse(x):
    res=np.zeros(Nparameters)
    for i in range(Nparameters):
        res[i]=x[i]*(ub[i] - lb[i]) *0.5 + (ub[i] + lb[i])*0.5
    return res

X_train_trafo = np.array([myscale(x) for x in X_train])
X_val_trafo   = np.array([myscale(x) for x in X_val])
X_test_trafo  = np.array([myscale(x) for x in X_test])
X_train_trafo = np.array([myscale(x) for x in X_train])
X_val_trafo   = np.array([myscale(x) for x in X_val])
X_test_trafo  = X_test_trafo.reshape((Ntest,5,1,1))
X_train_trafo = X_train_trafo.reshape((Ntrain,5,1,1))
X_val_trafo   = X_val_trafo.reshape((Nval,5,1,1))


In [3]:

#Neural Network
keras.backend.set_floatx('float64')
NN1 = Sequential() 
NN1.add(InputLayer(input_shape=(Nparameters,1,1,)))
NN1.add(ZeroPadding2D(padding=(2, 2)))
NN1.add(Conv2D(4, (3, 1), padding='valid',strides =(1,1),activation='elu'))#X_train_trafo.shape[1:],activation='elu'))
NN1.add(ZeroPadding2D(padding=(1,1)))
NN1.add(Conv2D(10, (3, 3),padding='valid',strides =(1,1),activation ='elu'))
NN1.add(MaxPooling2D(pool_size=(2, 1)))
#NN1.add(Dropout(0.25))
NN1.add(Conv2D(60, (2, 2),padding='valid',strides =(2,1),activation ='elu'))
NN1.add(Flatten())
NN1.add(Dense(Nstrikes*Nmaturities, activation = 'linear', kernel_constraint = keras.constraints.NonNeg()))
NN1.summary()

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
def root_relative_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square((y_pred - y_true)/y_true)))    
def root_relative_mean_squared_error_lasso(y_true, y_pred):
        return K.sqrt(K.mean(K.square((y_pred - y_true)/y_true)))+1/np.linalg.norm(y_pred)  
                
#NN1.compile(loss = root_relative_mean_squared_error, optimizer = "adam",metrics=root_mean_squared_error)
NN1.compile(loss = "mean_squared_error", optimizer = "adam")
#NN1.compile(loss = root_relative_mean_squared_error_lasso, optimizer = "adam",metrics=[root_relative_mean_squared_error,"mean_squared_error"])
#NN1.compile(loss = 'mean_absolute_percentage_error', optimizer = "adam")
NN1.fit(X_train_trafo, y_train_trafo, batch_size=32, validation_data = (X_val_trafo, y_val_trafo),
        epochs = 200, verbose = True, shuffle=1)
#NN1.save_weights('NN_HNGarch_weights.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_3 (ZeroPaddin (None, 9, 5, 1)           0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 5, 4)           16        
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 9, 7, 4)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 5, 10)          370       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 5, 10)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 4, 60)          2460      
_________________________________________________________________
flatten_2 (Flatten)          (None, 240)               0         
__________

NotFoundError: No registered '_MklConv2DBackpropFilterWithBias' OpKernel for CPU devices compatible with node {{node training_1/Adam/gradients/conv2d_5/convolution_grad/Conv2DBackpropFilter}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_DOUBLE, _kernel="MklOp", data_format="NHWC", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"
	.  Registered:  device='CPU'; label='MklOp'; T in [DT_FLOAT]

	 [[{{node training_1/Adam/gradients/conv2d_5/convolution_grad/Conv2DBackpropFilter}}]]